# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770857510647                   -0.52    9.0    174ms
  2   -2.772147933039       -2.89       -1.32    1.0    122ms
  3   -2.772170254419       -4.65       -2.44    1.0    102ms
  4   -2.772170647895       -6.41       -3.15    1.0    106ms
  5   -2.772170722761       -7.13       -4.25    2.0    124ms
  6   -2.772170722853      -10.04       -4.38    1.0    109ms
  7   -2.772170723012       -9.80       -5.29    1.0    116ms
  8   -2.772170723014      -11.71       -5.38    1.0    117ms
  9   -2.772170723015      -11.91       -6.00    1.0    118ms
 10   -2.772170723015      -12.78       -6.66    2.0    206ms
 11   -2.772170723015   +  -14.24       -6.99    1.0    862ms
 12   -2.772170723015   +  -15.35       -7.73    1.0    113ms
 13   -2.772170723015      -14.15       -8.07    1.0    113ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   --

1.7735579634498053

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770732543230                   -0.53   10.0    184ms
  2   -2.772049378019       -2.88       -1.31    1.0    132ms
  3   -2.772082358629       -4.48       -2.64    1.0    119ms
  4   -2.772083413786       -5.98       -3.79    2.0    123ms
  5   -2.772083417509       -8.43       -4.29    1.0    109ms
  6   -2.772083417785       -9.56       -5.15    1.0    113ms
  7   -2.772083417810      -10.62       -5.52    2.0    132ms
  8   -2.772083417811      -11.98       -6.67    1.0    115ms
  9   -2.772083417811      -13.75       -6.77    2.0    133ms
 10   -2.772083417811      -14.15       -7.07    1.0    115ms
 11   -2.772083417811   +  -14.18       -8.18    1.0    121ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.